In [6]:
!pip install mtcnn

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import sys

# from src.insightface.src.common import face_preprocess
from src.insightface.src.common import face_preprocess

sys.path.append('../insightface/deploy')
sys.path.append('../insightface/src/common')

from mtcnn.mtcnn import MTCNN
# import face_preprocess
import numpy as np
import cv2
import os
from datetime import datetime


class TrainingDataCollector:

    def __init__(self, args):
        self.args = args
        # Detector = mtcnn_detector
        self.detector = MTCNN()

    def collectImagesFromCamera(self):
        # initialize video stream
        cap = cv2.VideoCapture(0)

        # Setup some useful var
        faces = 0
        frames = 0
        max_faces = int(self.args["faces"])
        max_bbox = np.zeros(4)

        if not (os.path.exists(self.args["output"])):
            os.makedirs(self.args["output"])

        while faces < max_faces:
            ret, frame = cap.read()
            frames += 1

            dtString = str(datetime.now().microsecond)
            # Get all faces on current frame
            bboxes = self.detector.detect_faces(frame)

            if len(bboxes) != 0:
                # Get only the biggest face
                max_area = 0
                for bboxe in bboxes:
                    bbox = bboxe["box"]
                    bbox = np.array([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
                    keypoints = bboxe["keypoints"]
                    area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                    if area > max_area:
                        max_bbox = bbox
                        landmarks = keypoints
                        max_area = area

                max_bbox = max_bbox[0:4]

                # get each of 3 frames
                if frames % 3 == 0:
                    # convert to face_preprocess.preprocess input
                    landmarks = np.array([landmarks["left_eye"][0], landmarks["right_eye"][0], landmarks["nose"][0],
                                          landmarks["mouth_left"][0], landmarks["mouth_right"][0],
                                          landmarks["left_eye"][1], landmarks["right_eye"][1], landmarks["nose"][1],
                                          landmarks["mouth_left"][1], landmarks["mouth_right"][1]])
                    landmarks = landmarks.reshape((2, 5)).T
                    nimg = face_preprocess.preprocess(frame, max_bbox, landmarks, image_size='112,112')

                    cv2.imwrite(os.path.join(self.args["output"], "{}.jpg".format(dtString)), nimg)
                    cv2.rectangle(frame, (max_bbox[0], max_bbox[1]), (max_bbox[2], max_bbox[3]), (255, 0, 0), 2)
                    print("[INFO] {} Image Captured".format(faces + 1))
                    faces += 1
            cv2.imshow("Face detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'google.protobuf'

In [3]:
def collectImagesFromCamera(self):
        # initialize video stream
        cap = cv2.VideoCapture(0)

In [4]:
import logging


def get_logger():
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)s %(levelname)s \t%(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger


class ConfigurationsPOJO:
    camNo1 = 'cam1'
    camNo2 = 'cam2'
    camNo3 = 'cam3'
    camNo4 = 'cam4'
    TimeFormat = "%H:%M:%S"
    timezone = 'Asia/Calcutta'
    cameraIpLoc = 'rtsp:/admin:admin1234@192.168.1.15:554/ISAPI/Streaming/channels/{}/picture'

    # For DeepSort Tracker
    deepSortModelPath = "trackerModels/ckpt.t7"

    # For Yolo Object Detection
    yoloV3CfgPath = "detector/YOLOv3/cfg/yolo_v3.cfg"
    yoloV3WghtsPath = "trackerModels/yolov3.weights"
    yoloV3CocoNamesPath = "detector/YOLOv3/cfg/coco.names"

    # For DB Operations
    url = "mongodb://localhost:27017/"
    dbName = "userDetails"
    collectionName = "userAttendance"
    collectionNameForUnknwnUser = "userAttendanceForUnknwnUsr"

    # for Classifier Model path
    clssfr_ModelPath = 'faceEmbeddingModels/my_model.h5'



In [6]:
pip install src.com_in_ineuron_ai_detectfaces_mtcnn.mtcnn2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement src.com_in_ineuron_ai_detectfaces_mtcnn.mtcnn2 (from versions: none)
ERROR: No matching distribution found for src.com_in_ineuron_ai_detectfaces_mtcnn.mtcnn2

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install src.com_in_ineuron_ai_tracker_utils.tools

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement src.com_in_ineuron_ai_tracker_utils.tools (from versions: none)
ERROR: No matching distribution found for src.com_in_ineuron_ai_tracker_utils.tools

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import tensorflow as tf

from src.ai_detect_faces_mtcnn.mtcnn2 import PNet, RNet, ONet
from src.ai_tracker_utils.tools import detect_face, get_model_filenames


# class FaceDetectorMtcnN(threading.Thread):
class FaceDetectorMtcnN():
    def __init__(self, args):
        # threading.Thread.__init__(self)
        self.file_paths = get_model_filenames(args.model_dir)
        self.minsize = args.minsize
        self.threshold = args.thresholdVal
        self.factor = args.factor
        self.save_image = args.save_image
        self.save_name = args.save_name
        # self.img_path = args.image_path

    def run(self, img):
        # img = cv2.imread(self.img_path)
        with tf.device('/gpu:0'):
            with tf.Graph().as_default():
                config = tf.ConfigProto(allow_soft_placement=True)
                with tf.Session(config=config) as sess:
                    if len(self.file_paths) == 3:
                        image_pnet = tf.placeholder(
                            tf.float32, [None, None, None, 3])
                        pnet = PNet({'data': image_pnet}, mode='test')
                        out_tensor_pnet = pnet.get_all_output()

                        image_rnet = tf.placeholder(tf.float32, [None, 24, 24, 3])
                        rnet = RNet({'data': image_rnet}, mode='test')
                        out_tensor_rnet = rnet.get_all_output()

                        image_onet = tf.placeholder(tf.float32, [None, 48, 48, 3])
                        onet = ONet({'data': image_onet}, mode='test')
                        out_tensor_onet = onet.get_all_output()

                        saver_pnet = tf.train.Saver(
                                        [v for v in tf.global_variables()
                                         if v.name[0:5] == "pnet/"])
                        saver_rnet = tf.train.Saver(
                                        [v for v in tf.global_variables()
                                         if v.name[0:5] == "rnet/"])
                        saver_onet = tf.train.Saver(
                                        [v for v in tf.global_variables()
                                         if v.name[0:5] == "onet/"])

                        saver_pnet.restore(sess, self.file_paths[0])

                        def pnet_fun(img): return sess.run(
                            out_tensor_pnet, feed_dict={image_pnet: img})

                        saver_rnet.restore(sess, self.file_paths[1])

                        def rnet_fun(img): return sess.run(
                            out_tensor_rnet, feed_dict={image_rnet: img})

                        saver_onet.restore(sess, self.file_paths[2])

                        def onet_fun(img): return sess.run(
                            out_tensor_onet, feed_dict={image_onet: img})

                    else:
                        saver = tf.train.import_meta_graph(self.file_paths[0])
                        saver.restore(sess, self.file_paths[1])

                        def pnet_fun(img): return sess.run(
                            ('softmax/Reshape_1:0',
                             'pnet/conv4-2/BiasAdd:0'),
                            feed_dict={
                                'Placeholder:0': img})

                        def rnet_fun(img): return sess.run(
                            ('softmax_1/softmax:0',
                             'rnet/conv5-2/rnet/conv5-2:0'),
                            feed_dict={
                                'Placeholder_1:0': img})

                        def onet_fun(img): return sess.run(
                            ('softmax_2/softmax:0',
                             'onet/conv6-2/onet/conv6-2:0',
                             'onet/conv6-3/onet/conv6-3:0'),
                            feed_dict={
                                'Placeholder_2:0': img})

                    # start_time = time.time()
                    bounding_boxes = detect_face(img, self.minsize,
                                                     pnet_fun, rnet_fun, onet_fun,
                                                     self.threshold, self.factor)
        return bounding_boxes
                    

ModuleNotFoundError: No module named 'src.com_in_ineuron_ai_detectfaces_mtcnn'